In [ ]:
pip install gradio==5.12.0

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\noebo\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [2]:
import pandas as pd
from matplotlib import pyplot as plt
import gradio as gr

C:\Users\noebo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
###### IMPORTATION DES DATASETS #######


# initialisation des listes
data_list = []
grouped_list = []
grouped_dep_list = []

# importation des datasets et éventuelles uniformisations
data_offre_complete = pd.read_csv('Données pour Dashboard\offre_complete.csv')
data_offre_complete.replace(" ", "_", regex=True, inplace=True)

data_rev_dep = pd.read_csv('Données pour Dashboard\data_rev_dep.csv')
data_rev_dep.replace(" ", "_", regex=True, inplace=True)
data_depenses = pd.read_csv('Données pour Dashboard\depenses-culturelles-des-departements-total.csv')

# jointures
data_complet = pd.merge(data_offre_complete, data_depenses, how='left', on='DEPARTEMENT')
data_complet_revenus = pd.merge(data_offre_complete, data_rev_dep, how='left', on='DEPARTEMENT')

# importation du reste des datasets
data_musees = pd.read_csv('Données pour Dashboard\data_musees.csv')
data_festivals = pd.read_csv('Données pour Dashboard\data_festivals.csv')
data_batmon = pd.read_csv('Données pour Dashboard\\NewType_Bat.csv')
data_theatre = pd.read_csv('Données pour Dashboard\data_theatre.csv')
data_cinema = pd.read_csv('Données pour Dashboard\data_cinema.csv')
data_biblio = pd.read_csv('Données pour Dashboard\data_biblio.csv')
data_archives = pd.read_csv('Données pour Dashboard\data_archives.csv')

# création des dataframes "groupés" selon la région et le type
grouped_musees = data_musees.groupby(['nomReg', 'DOMAINE_SOUS_TYPE']).size().unstack(fill_value=0)
grouped_festivals = data_festivals.groupby(['nomReg', 'Type']).size().unstack(fill_value=0)
grouped_batmon = data_batmon.groupby(['nomReg', 'TYPE']).size().unstack(fill_value=0)
grouped_theatre = data_theatre.groupby(['nomReg', 'SOUS_TYPE']).size().unstack(fill_value=0)
grouped_cinema = data_cinema.groupby(['nomReg', 'Type']).size().unstack(fill_value=0)
grouped_biblio = data_biblio.groupby(['nomReg', 'SOUS_TYPE_STATUT']).size().unstack(fill_value=0)
grouped_archives = data_archives.groupby(['nomReg', 'Type']).size().unstack(fill_value=0)

# idem selon la région et le type
grouped_musees_dep = data_musees.groupby(['nomdept', 'DOMAINE_SOUS_TYPE']).size().unstack(fill_value=0)
grouped_festivals_dep = data_festivals.groupby(['nomdept', 'Type']).size().unstack(fill_value=0)
grouped_batmon_dep = data_batmon.groupby(['nomdept', 'TYPE']).size().unstack(fill_value=0)
grouped_theatre_dep = data_theatre.groupby(['nomdept', 'SOUS_TYPE']).size().unstack(fill_value=0)
grouped_cinema_dep = data_cinema.groupby(['nomdept', 'Type']).size().unstack(fill_value=0)
grouped_biblio_dep = data_biblio.groupby(['nomdept', 'SOUS_TYPE_STATUT']).size().unstack(fill_value=0)
grouped_archives_dep = data_archives.groupby(['nomdept', 'Type']).size().unstack(fill_value=0)

# ajout des dataframes dans data_list
data_list.append(data_musees)
data_list.append(data_festivals)
data_list.append(data_batmon)
data_list.append(data_theatre)
data_list.append(data_cinema)
data_list.append(data_biblio)
data_list.append(data_archives)

# ajout des datafames groupés dans grouped_list
grouped_list.append(grouped_musees)
grouped_list.append(grouped_festivals)
grouped_list.append(grouped_batmon)
grouped_list.append(grouped_theatre)
grouped_list.append(grouped_cinema)
grouped_list.append(grouped_biblio)
grouped_list.append(grouped_archives)

# ajout des dataframes groupés (départements) dans grouped_dep_list
grouped_dep_list.append(grouped_musees_dep)
grouped_dep_list.append(grouped_festivals_dep)
grouped_dep_list.append(grouped_batmon_dep)
grouped_dep_list.append(grouped_theatre_dep)
grouped_dep_list.append(grouped_cinema_dep)
grouped_dep_list.append(grouped_biblio_dep)
grouped_dep_list.append(grouped_archives_dep)


# index pour retrouver la position des dataframes selon le type
index_type = ['Musées' , 'Festivals', 'Bâtiments/Monuments',
                       'Théâtres', 'Cinémas', 'Bibliothèques', 'Archives']





In [4]:
##### FONCTIONS OUTILS #####

# fonction qui renvoie le maximum d'une liste (cas où il n'y a pas que des int)
def maxi(liste):
    maximum = 0
    for elem in liste:
        if isinstance(elem, int):
            maximum = max(maximum, elem)
    return maximum

# fonction (et liste) qui uniformise une chaîne de caractères
list_of_changes = [
    (' *', ''),
    ('É', 'E'),
    ('È', 'E'),
    ('Ê', 'E'),
    ('Ë', 'E'),
    ('Á', 'A'),
    ('Â', 'A'),
    ('À', 'A'),
    ('Ô', 'O'),
    ('Ù', 'U'),
    ('Î', 'I'),
    ('Ï', 'I'),
    (' ', '_'),
    ("\'", '_'),
    (',', ''),
    ('’', '_')
                   ]
def replace_multiple(string, l):
    for i in range(len(l)):
        string = string.replace(l[i][0], l[i][1])
    return string


# fonction qui permet d'afficher le pourcentage et la valeur absolue pour
# les diagrammes en camembert
def func(pct, allvalues):
    absolute = int(pct / 100. * sum(allvalues))
    return "{:.1f}%\n({:d} euros/an)".format(pct, absolute)


# fonction qui renvoie les départements d'une région
def dropdown_dep(region):
    dep_list = []
    for data in data_list:
        data = data[data.nomReg == region]
        dep_list.append(list(data.nomdept))
    return list(set(dep_list[0]))


# fonction qui renvoie un dropdown avec les départements de la région passée en
# paramètre
def update_page_2(region):
    return gr.Dropdown(
            label="Sélectionnez les Départements :",
            choices=dropdown_dep(region),
            value = dropdown_dep(region),
        )

##### FONCTIONS GENERATRICES ######

# fonction qui génère les graphiques du premier onglet
def gen_barplot(offre, region):

    ## manipulations du dataframe pour afficher les proportions

    # on récupère le dataframe de l'offre choisie
    i = index_type.index(offre)
    grouped = grouped_list[i]
    data = data_list[i]

    # si aucune région n'est choisie, on prend toutes les régions
    if len(region) > 0:
        grouped = grouped[(grouped.index.isin(region))]
        data = data[(data.nomReg.isin(region))]

    columns_list = grouped.columns.values.tolist()

    grouped_final = grouped.div(grouped.sum(axis=1), axis=0)


    ## création de la première figure à retourner

    fig_prop, ax_prop = plt.subplots(figsize=(8, 6))
    palette = plt.cm.tab20(range(20))
    ax_prop = grouped_final.plot(kind='bar', stacked=True, ax=ax_prop, color=palette)
    plt.xticks(rotation=45, ha='right')
    ax_prop.set_xlabel("Nom de la Région")
    ax_prop.set_ylabel("Proportion des types d'offres culturelles")
    ax_prop.legend(bbox_to_anchor=(1.01, 1), loc=2, borderaxespad=0., labels=columns_list)
    fig_prop.tight_layout()


    ## manipulations du dataframe pour afficher les nombre d'offres par région

    data_count = data['nomReg'].value_counts().reset_index()
    data_count.columns = ['region', 'count']


    ## création de la deuxième figure à retourner

    fig_bar, ax_bar = plt.subplots(figsize=(8, 6))
    ax_bar.bar(data_count['region'], data_count['count'], color=palette)
    ax_bar.set_xlabel("Nom de la Région", fontsize=12)
    ax_bar.set_ylabel("Nombre d'offres culturelles", fontsize=12)
    plt.xticks(rotation=45, ha='right')
    fig_bar.tight_layout()


    return fig_prop, fig_bar, str(grouped.values.sum()) + " / 68 084"


# fonction qui génère les graphiques du deuxième onglet
def generer_graphiques_page2(region, departements):
    palette = plt.cm.tab20(range(20))

    ## uniformisation des départements
    for i in range(len(departements)):
        departements[i] = replace_multiple(departements[i], list_of_changes)

    ### Diagrammes à camembert ###

    ## création des dataframes à manipuler
    data_dep = data_depenses[data_depenses["DEPARTEMENT"].isin(departements)]
    data_dep = data_dep.sort_values(by=['DEPARTEMENT'], ascending=True)

    ## création de la figure pour les dépenses
    fig_depenses, ax_depenses = plt.subplots(figsize=(8, 8))
    ax_depenses.pie(
        data_dep.DEPENSES_TOT,
        labels=data_dep.DEPARTEMENT.values,
        autopct='%1.1f%%',
        startangle=90,
        colors=palette
    )
    fig_depenses.tight_layout()

    ## création de la figure pour les dépenses par habitant
    fig_dep_hab, ax_effort = plt.subplots(figsize=(8, 8))
    ax_effort.pie(
        data_dep.DEPENSES_TOT_HAB,
        labels=data_dep.DEPARTEMENT.values,
        autopct='%1.1f%%',
        startangle=90,
        colors=palette
    )
    fig_dep_hab.tight_layout()

    ## création du dataframe pour les revenus moyens
    data = data_rev_dep[data_rev_dep.DEPARTEMENT.isin(departements)]
    data = data[data.DEPARTEMENT != 'PARIS']
    data = data.sort_values(by=['DEPARTEMENT'], ascending=True)

    ## création de la figure pour les revenus moyens
    fig_rev, ax_rev = plt.subplots(figsize=(8, 8))
    ax_rev.pie(
        data.REVENU_MOYEN,
        labels=data.DEPARTEMENT.values,
        autopct=lambda pct: func(pct, data.REVENU_MOYEN),
        startangle=90,
        colors=palette
    )
    fig_rev.tight_layout()


    ### Courbes ###

    ## manipulations première courbe

    # on garde que les départements concernés
    data = data_complet[data_complet.DEPARTEMENT.isin(departements)]
    data = data[data.DEPARTEMENT != 'PARIS']

    # on trie les départements par les dépenses
    order_departements = data.sort_values('DEPENSES_TOT', ascending=True).DEPARTEMENT.unique().tolist()


    # on garde les dépenses totales par département
    grouped = data.groupby(['DEPENSES_TOT', 'DEPARTEMENT']).count().reset_index()
    grouped.sort_values(by=['DEPENSES_TOT'], ascending=True)

    ## création première courbe
    fig_courbe_dep, ax_courbe_dep = plt.subplots(figsize=(8, 8))
    ax_courbe_dep.scatter(
        grouped.DEPENSES_TOT.values,
        grouped.NOM,
    )
    for i in range(len(grouped)):
        ax_courbe_dep.scatter(
            grouped.DEPENSES_TOT[i],
            grouped.NOM[i],
            c=palette[i],
            label=order_departements[i]
        )

    ax_courbe_dep.legend()
    ax_courbe_dep.set_xlabel("Dépenses totales annuelles (dizaines de millions d'euros)")
    ax_courbe_dep.set_ylabel("Nombre d'offres culturelles")
    fig_courbe_dep.tight_layout()


    ## manipulations deuxième courbe

    # on trie les départements par les dépenses par habitant
    order_departements = data.sort_values('DEPENSES_TOT_HAB', ascending=True).DEPARTEMENT.unique().tolist()

    # on garde les dépenses par habitant par département
    grouped = data.groupby(['DEPENSES_TOT_HAB', 'DEPARTEMENT']).count().reset_index()
    grouped.sort_values(by=['DEPENSES_TOT_HAB'], ascending=True)

    # création deuxième courbe
    fig_courbe_hab, ax_courbe_hab = plt.subplots(figsize=(8, 8))
    for i in range(len(grouped)):
        ax_courbe_hab.scatter(
            grouped.DEPENSES_TOT_HAB[i],
            grouped.NOM[i],
            c=palette[i],
            label=order_departements[i]
        )

    ax_courbe_hab.legend()
    ax_courbe_hab.set_xlabel('Dépenses par habitant (euros)')
    ax_courbe_hab.set_ylabel("Nombre d'offres culturelles")
    fig_courbe_hab.tight_layout()


    ## manipulations troisième courbe

    # on garde que les départements concernés
    data = data_complet_revenus[data_complet_revenus.DEPARTEMENT.isin(departements)]
    data = data[data.DEPARTEMENT != 'PARIS']

    # on trie les départements par l'effort budgétaire total
    order_departements = data.sort_values('REVENU_MOYEN', ascending=True).DEPARTEMENT.unique().tolist()

    # on garde les revenus moyens par département
    grouped = data.groupby(['REVENU_MOYEN', 'DEPARTEMENT']).count().reset_index()

    ## création troisième courbe

    fig_courbe_revenu, ax_courbe_revenu = plt.subplots(figsize=(8, 8))


    for i in range(len(grouped)):
        ax_courbe_revenu.scatter(
            grouped.REVENU_MOYEN[i],
            grouped.NOM[i],
            c = palette[i],
            label = order_departements[i]
        )

    ax_courbe_revenu.legend()

    ax_courbe_revenu.set_xlabel("Revenu annuel moyen par foyer (euros)")
    ax_courbe_revenu.set_ylabel("Nombre d'offres")
    plt.xticks(rotation=45)
    plt.tight_layout()

    return fig_depenses, fig_dep_hab, fig_rev, fig_courbe_dep, fig_courbe_hab, fig_courbe_revenu



# fonction qui génère les grahiques du troisième onglet
def generer_stat_Musee(NomReg,type_visite) :
    typeent = ''
    if type_visite == 'Gratuits':
        typeent = "ENTRE_GRATUIT"
    elif type_visite == 'Total':
        typeent = "ENTRE_TOTAL"
    elif type_visite == 'Payants':
        typeent = "ENTRE_PAYANT"

    ## sélection du dataframe concerné
    data_museesS = data_musees[data_musees.nomReg.isin(NomReg)]
    data_museesG = data_museesS.sort_values(by=typeent, ascending=False).head(10)

    ## création de la figure à retourner
    colors = ['#add8e6', '#87ceeb', '#6495ed', '#4682b4', '#4169e1',
                '#6a5acd', '#7b68ee', '#9370db', '#8a2be2', '#9400d3']
    figM, axM = plt.subplots(figsize=(14, 6))
    bars = axM.bar(range(len(data_museesG)), data_museesG[typeent], color=colors)
    axM.set_xticks([])
    i = 1
    for bar, name, departement, color in zip(bars, data_museesG["NOM"], data_museesG['nomdept'], colors):
        bar.set_label(str(i) + " - " + name.title() + " (" + departement + ")")  # Associer chaque barre à un nom
        i += 1
    axM.legend(title="Classement", loc=2, fontsize=9, title_fontsize=10,bbox_to_anchor=(1.01, 1), borderaxespad=0.)
    axM.set_title(f"Classement des musées les plus visités ({type_visite.lower()})")
    axM.set_xlabel("Musées")
    axM.set_ylabel("Nombre de visites (en millions)")
    figM.tight_layout()
    return  figM

def generer_stat_Cinema(NomReg,type) :

    if type == 'Par nombre de salles':
        typeent = 'ecran'
        type2 = "salles"
    elif type == 'Par nombre de places':
        typeent = 'fauteuil'
        type2 = "places"

    ## sélection du dataframe concerné
    data = data_cinema[data_cinema.nomReg.isin(NomReg)]
    data = data.sort_values(by=typeent.upper(), ascending=False).head(10)

    ## création de la figure à retourner
    colors = ['#add8e6', '#87ceeb', '#6495ed', '#4682b4', '#4169e1',
                '#6a5acd', '#7b68ee', '#9370db', '#8a2be2', '#9400d3']
    figM, axM = plt.subplots(figsize=(12, 6))
    bars = axM.bar(range(len(data)), data[typeent.upper()], color=colors)
    axM.set_xticks([])
    i = 1
    for bar, name, departement, color in zip(bars, data["NOM"], data['nomdept'], colors):
        bar.set_label(str(i) + " - " +name.title() + " (" + departement + ")")  # Associer chaque barre à un nom
        i += 1
    axM.legend(title="Classement", loc=2, fontsize=9, title_fontsize=10,bbox_to_anchor=(1.01, 1), borderaxespad=0.)
    axM.set_title(f"Classement des cinémas par {type2}")
    axM.set_xlabel("Cinémas")
    axM.set_ylabel(f"Nombre de {type2}")
    figM.tight_layout()
    return  figM



def generer_stat_biblio(NomReg,typec) :
    typeent = ""
    if typec=='Nombre de salariés':
      typeent='NB_SALARIE'
    elif typec=='Nombre de bénévoles' :
      typeent='NB_BENEVOLE'
    elif typec== 'Surface' :
      typeent='SURFACE'

    ## sélection du dataframe concerné
    data = data_biblio[data_biblio.nomReg.isin(NomReg)]
    data = data.sort_values(by=typeent.upper(), ascending=False).head(10)

    ## création de la figure à retourner
    colors = ['#add8e6', '#87ceeb', '#6495ed', '#4682b4', '#4169e1',
              '#6a5acd', '#7b68ee', '#9370db', '#8a2be2', '#9400d3']
    figM, axM = plt.subplots(figsize=(12, 6))
    bars = axM.bar(range(len(data)), data[typeent.upper()], color=colors)
    axM.set_xticks([])
    i = 1
    for bar, name, departement, color in zip(bars, data["NOM"], data['nomdept'], colors):
        bar.set_label(str(i) + " - " + name.title() + " (" + departement + ")")  # Associer chaque barre à un nom
        i += 1
    axM.legend(title="Classement", loc=2, fontsize=9, title_fontsize=10,bbox_to_anchor=(1.01, 1), borderaxespad=0.)
    axM.set_title(f"Classement des bibliothèques par {typec.lower()}")
    axM.set_xlabel(typec)
    axM.set_ylabel("Bibliothèque")
    figM.tight_layout()

    return figM


# initialisation de la liste des départements sélectionnés
deps = []

# on prend les différentes régions (sauf celles où il y a des données manquantes)
regions = data_musees.nomReg.unique().tolist()
regions.remove('Martinique')
regions.remove('Guyane')
regions.remove('Corse')
regions.remove('La Réunion')
regions.remove('Mayotte')
regions.remove('Guadeloupe')


# css pour le titre
title=(
"""
<center>

<h1> Étude des différentes offres culturelles en France </h1>

</center>
"""
)

##### GRADIO ######
with gr.Blocks(css="""
    .custom-dropdown {
        border: 1px solid grey !important;
        border-radius: 5px;
        color: #F5f5dc;
        background-color : #cccccc36;
    }
    .graph{
        border: 1px solid grey !important;
        border-radius: 10px;
        color: #F5f5dc;
               }
    .KPI{
      border: 1px solid grey !important;
        border-radius: 10px;
        color: #F5f5dc;
        background-color : #6ee7b7;

    }
""", theme=gr.themes.Ocean()) as demo:
    with gr.Row():
        gr.HTML(title)

    with gr.Tab("Chiffres clés par Région", elem_classes="custom-dropdown"):

        offre_input = gr.Radio(
            label = "Sélection de l'offre culturelle :",
            choices = index_type,
        )

        departement_input = gr.Dropdown(
            label="Sélectionnez les Régions :",
            choices= data_musees.nomReg.unique().tolist(),
            value = data_musees.nomReg.unique().tolist(),
            multiselect=True,
        )

        with gr.Row():
            with gr.Column():
                pass
            with gr.Column():
                kpi_output = gr.Textbox(
                    label="KPI : Nombre d'offres culturelles", elem_classes="KPI"
                )
            with gr.Column():
                pass


        with gr.Row():

            with gr.Column():
                gr.Markdown("### Nombre d'offres culturelles par région")
                output_graph_bar = gr.Plot(show_label=False, elem_classes="graph")
            with gr.Column():
                gr.Markdown("### Proportion des offres culturelles par région")
                output_graph_prop = gr.Plot(show_label=False, elem_classes="graph")

        offre_input.select(
            fn = gen_barplot,
            inputs=[offre_input, departement_input],
            outputs=[output_graph_prop, output_graph_bar, kpi_output]
        )

        departement_input.change(
            fn = gen_barplot,
            inputs=[offre_input, departement_input],
            outputs=[output_graph_prop, output_graph_bar, kpi_output]
        )

    with gr.Tab("Analyses socio-économiques", elem_classes="custom-dropdown"):
        region_input = gr.Radio(
            label="Sélection de la Région :",
            choices=regions,
        )

        dep_input = gr.Dropdown(
            label="Sélectionnez les Départements :",
            choices=deps,
            interactive=True,
            multiselect=True
        )
        with gr.Row():
            with gr.Column():
                gr.Markdown("### Dépenses culturelles par département")
                output_graph_depenses = gr.Plot(show_label=False, elem_classes="graph")
            with gr.Column():
                gr.Markdown("### Dépenses culturelles par habitant par département")
                output_graph_budget = gr.Plot(show_label=False, elem_classes="graph")
            with gr.Column():
                gr.Markdown("### Revenu moyen par foyer par département")
                output_graph_revenu = gr.Plot(show_label=False, elem_classes="graph")

        with gr.Row():
            with gr.Column():
                gr.Markdown("### Dépenses totales annuelles selon le nombre d'offres culturelles disponibles")
                output_graph_courbe_depenses = gr.Plot(show_label=False, elem_classes="graph")
            with gr.Column():
                gr.Markdown("### Dépenses culturelles par habitant selon le nombre d'offres culturelles disponibles")
                output_graph_courbe_budget = gr.Plot(show_label=False, elem_classes="graph")
            with gr.Column():
                gr.Markdown("### Revenu moyen par foyer selon le nombre d'offres culturelles disponibles")
                output_graph_courbe_revenus = gr.Plot(show_label=False, elem_classes="graph")


        region_input.change(
            fn=update_page_2,
            inputs=[region_input],
            outputs=[dep_input],
        )


        dep_input.change(
            fn=generer_graphiques_page2,
            inputs=[region_input, dep_input],
            outputs=[output_graph_depenses, output_graph_budget, output_graph_revenu,
                     output_graph_courbe_depenses, output_graph_courbe_budget, output_graph_courbe_revenus],
        )

        demo.load(
        )

    with gr.Tab("Classement des Musées par Région", elem_classes="custom-dropdown"):
        offre_input = gr.Radio(
            label="Sélection du type d'entrée",
            choices=["Gratuits", "Payants", "Total"]
        )
        departement_input = gr.Dropdown(
            label="Sélectionnez les Régions :",
            choices=data_musees.nomReg.unique().tolist(),
            multiselect=True,
            elem_classes="custom-dropdown",
            value=data_musees.nomReg.unique().tolist(),
        )

        with gr.Row():
            bar_output = gr.Plot(show_label=False, elem_classes="graph")

            offre_input.select(
                fn=generer_stat_Musee,
                inputs=[departement_input, offre_input],
                outputs=[bar_output]
            )

            departement_input.change(
                fn=generer_stat_Musee,
                inputs=[departement_input, offre_input],
                outputs=[bar_output]
            )

    with gr.Tab("Classement des Cinémas par Région", elem_classes="custom-dropdown"):

        offre_input = gr.Radio(
            label="Sélection du type de classement",
            choices=["Par nombre de salles", "Par nombre de places"]
        )
        departement_input = gr.Dropdown(
            label="Sélectionnez les Régions :",
            choices=data_musees.nomReg.unique().tolist(),
            multiselect=True,
            elem_classes="custom-dropdown",
            value=data_musees.nomReg.unique().tolist(),
        )



        with gr.Row():
            bar_output = gr.Plot(show_label=False, elem_classes="graph")

            offre_input.select(
                fn=generer_stat_Cinema,
                inputs=[departement_input, offre_input],
                outputs=[bar_output]
            )

            departement_input.change(
                fn=generer_stat_Cinema,
                inputs=[departement_input, offre_input],
                outputs=[bar_output]
            )

    with gr.Tab("Classement des Bibliothèques par Région", elem_classes="custom-dropdown"):

        trie_input = gr.Radio(
            label="Sélection du type de classement",
            choices=["Nombre de salariés", "Nombre de bénévoles", "Surface"]
        )

        departement_input = gr.Dropdown(
            label="Sélectionnez les Régions :",
            choices=data_musees.nomReg.unique().tolist(),
            multiselect=True,
            elem_classes="custom-dropdown",
            value=data_musees.nomReg.unique().tolist(),
        )


        with gr.Row():
            bar_output = gr.Plot(show_label=False, elem_classes="graph")

            trie_input.select(
                fn=generer_stat_biblio,
                inputs=[departement_input, trie_input],
                outputs=[bar_output]
            )

            departement_input.change(
                fn=generer_stat_biblio,
                inputs=[departement_input, trie_input],
                outputs=[bar_output]
            )

demo.launch(share=True)

* Running on local URL:  http://127.0.0.1:7860

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


C:\Users\noebo\AppData\Local\Temp\ipykernel_9372\3766039002.py:190: UserWarning: *c* argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with *x* & *y*.  Please use the *color* keyword-argument or provide a 2D array with a single row if you intend to specify the same RGB or RGBA value for all points.
  ax_courbe_dep.scatter(
C:\Users\noebo\AppData\Local\Temp\ipykernel_9372\3766039002.py:215: UserWarning: *c* argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with *x* & *y*.  Please use the *color* keyword-argument or provide a 2D array with a single row if you intend to specify the same RGB or RGBA value for all points.
  ax_courbe_hab.scatter(
C:\Users\noebo\AppData\Local\Temp\ipykernel_9372\3766039002.py:246: UserWarning: *c* argument looks like a single numeric RGB or RGBA sequence, which shou